In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import csv
import re
import time

In [ ]:
driver = webdriver.Chrome()
# departDate: 2019-08-30 from NYC; arrivalDate: 2019-09-02 to LAX
driver.get("https://www.kayak.com/flights/NYC-LAX/2019-08-30/2019-09-02?sort=bestflight_a")

In [ ]:
options = webdriver.ChromeOptions()
options.add_experimental_option('w3c', False)
create_webdriver('Chrome', options=options)

In [ ]:
def waitobject(xpath):
    wait_object = WebDriverWait(driver, 10)
    elements = wait_object.until(EC.presence_of_all_elements_located((By.XPATH, xpath)))
    return elements

#### click on the "Quickest" tab

In [ ]:
quick = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//a[@data-code = "duration"]')))
quick.click()

In [ ]:
def show_more_results():
    try:        
        showMoreResults_button = driver.find_element_by_xpath('//a[@class="moreButton"]')
        showMoreResults_button
        time.sleep(1)
    except:
        pass

In [ ]:
def scrape_flightPage():
    #find prices
    prices = driver.find_elements_by_xpath('//span[@class="price option-text"]')
    prices_list = [tag.text.replace('$','') for tag in prices if tag.text != '']
    prices_list = list(map(int, prices_list))

    #find number of stops
    stops = driver.find_elements_by_xpath('//div[@class="section stops"]/div[1]')
    stops_list = [tag.text[0] for tag in stops if stops != '']
    stops_list = [tag.replace('n','0') for tag in stops_list]
    stops_list = list(map(int, stops_list))
    source_stops_list = stops_list[0::2]
    destination_stops_list = stops_list[1::2]

    #find airport cities for stops
    airport_stops = driver.find_elements_by_xpath('//div[@class="section stops"]/div[2]')
    airport_stops_list = [tag.text for tag in airport_stops if airport_stops != '']
    source_airport_stops_list = airport_stops_list[0::2]
    destination_airport_stops_list = airport_stops_list[1::2]
    
    #find durations
    durations = driver.find_elements_by_xpath('//div[@class="section duration"]/div[1]')
    durations_list = [tag.text for tag in durations]
    source_durations_list = durations_list[0::2]
    destination_durations_list = durations_list[1::2]
    
    #find airports cities for source and destination
    airports = driver.find_elements_by_xpath('//div[@class="section duration"]/div[2]')
    airports_list = [tag.text for tag in airports]
    source_airports_list = airports_list[0::2]
    destination_airports_list = airports_list[1::2]

    #find the time
    times = driver.find_elements_by_xpath('//div[@class="section times"]/div[1]')
    times_list = [tag.text for tag in times]
    source_times_list = times_list[0::2]
    destination_times_list = times_list[1::2]
    
    #find carriers
    carriers = driver.find_elements_by_xpath('//div[@class="section times"]/div[2]')
    carriers_list = [tag.text for tag in carriers]
    source_carriers_list = carriers_list[0::2]
    destination_carriers_list = carriers_list[1::2]
    
    kayak_dict = {}

    kayak_dict['price'] = prices_list
    kayak_dict['source stops'] = source_stops_list
    kayak_dict['destination stops'] = destination_stops_list
    kayak_dict['source airport stops'] =  source_airport_stops_list
    kayak_dict['destination airport stops'] = destination_airport_stops_list
    kayak_dict['source duration'] = source_durations_list
    kayak_dict['destination duration'] = destination_durations_list
    kayak_dict['source airport'] = source_airports_list
    kayak_dict['destination airport'] = destination_airports_list
    kayak_dict['source time'] = source_times_list
    kayak_dict['destination time'] = destination_times_list
    kayak_dict['source carrier'] = source_carriers_list
    kayak_dict['destination carries'] = destination_carriers_list
    
    #df = pd.DataFrame(kayak_dict)
    
    return kayak_dict

#### click on the "Cheapest" tab

In [ ]:
cheapest_button = driver.find_element_by_xpath('//a[@data-code = "price"]')
cheapest_button.click()

In [ ]:
def scrape_data(source, destination, start_date, end_date):
    driver.get( 'https://www.kayak.com/flights/' + 
            source + '-' + destination + 
            '/' + start_date + '/' + end_date + '?sort=bestflight_a' )
    
    #close pop-up
    try:
        poUp_button = driver.find_elements_by_xpath('//button[contains(@id, "dialog-close") and contains(@class, "Button-No-Standard-Style close ")]')
        popUp_button[5].click()
    except:
        pass
    
    #show more results per page
#     show_more_results()
#     show_more_results()
 
    #scrape best flights page
    df_best = scrape_flightPage()
    df_best['sort'] = 'best'
    
    #click on the cheapest
    cheapest_button = driver.find_element_by_xpath('//a[@data-code = "price"]')
    cheapest_button.click()
    
    #show more results per page
    #show_more_results()
    #show_more_results()
    
    #scrape cheapest flights page
    df_cheap = page_scrape()
    df_cheap['sort'] = 'cheapest'


    #click on the quickest
    quickest_button = driver.find_element_by_xpath('//a[@data-code = "duration"]')
    quickest_button.click()
        
    #show more results per page
    #show_more_results()
    #show_more_results()
    
    #scrape quickest flights page
    df_quick = page_scrape()
    df_quick['sort'] = 'quickest'
    time.sleep(1)
    
    df = df_cheap.append(df_best).append(df_quick)
    
    # SAVE the WEBSCAPING DATA TO CSV FILE:
    df.to_csv('kayak.csv')
    [csv.writer('kayak.csv') for row in row]

    csv_file = open('kayak.csv', 'w', encoding='utf-8', newline=';')
    csv_writer = csv.writer(csv_file)

In [ ]:
class KayakScraper(object):
    def __init__(self):
        self.driver = webdriver.Chrome()
        self.file = open("Desktop/NYCDSA/projects/webscrape/kayak/data/kayak.csv",'w+')
        self.fieldnames = ['price','src_stops','dest_stops',
                    'src_airport_stops', 'dest_airport_stops',
                    'src_duration', 'dest_duration',
                    'src_airport', 'dest_airport',
                    'src_time', 'dest_time',
                    'src_carrier', 'dest_carrier',
                    'sort'
                ]

        ## Dictionary to CSV writer class
        self.writer = csv.DictWriter(self.file,
                fieldnames = self.fieldnames)
        ## Write column names to CSV header
        self.writer.writeheader()

        self.driver.get("https://www.kayak.com/")
        time.sleep(5)

#     def __del__(self):
#         self.driver.quit()
#         del self.writer
#         self.file.close()
        


    def show_more_results(self):
        try:        
            showMoreResults_button = self.driver.find_element_by_xpath('//a[@class="moreButton"]')
            showMoreResults_button
            time.sleep(1)
        except:
            pass


    def scrape_data(self, _sort = "best"):
        if _sort == "cheapest":
            self.clickCheapest()
        elif _sort == "quickest":
            self.clickQuickest()
        
        #find prices
        prices = self.driver.find_elements_by_xpath('//span[@class="price option-text"]')
        prices_list = [tag.text.replace('$','') for tag in prices if tag.text != '']
        prices_list = list(map(int, prices_list))

        #find number of stops
        stops = self.driver.find_elements_by_xpath('//div[@class="section stops"]/div[1]')
        stops_list = [tag.text[0] for tag in stops if stops != '']
        stops_list = [tag.replace('n','0') for tag in stops_list]
        stops_list = list(map(int, stops_list))
        source_stops_list = stops_list[0::2]
        destination_stops_list = stops_list[1::2]

        #find airport cities for stops
        airport_stops = self.driver.find_elements_by_xpath('//div[@class="section stops"]/div[2]')
        airport_stops_list = [tag.text for tag in airport_stops if airport_stops != '']
        source_airport_stops_list = airport_stops_list[0::2]
        destination_airport_stops_list = airport_stops_list[1::2]
        
        #find durations
        durations = self.driver.find_elements_by_xpath('//div[@class="section duration"]/div[1]')
        durations_list = [tag.text for tag in durations]
        source_durations_list = durations_list[0::2]
        destination_durations_list = durations_list[1::2]
        
        #find airports cities for source and destination
        airports = self.driver.find_elements_by_xpath('//div[@class="section duration"]/div[2]')
        airports_list = [tag.text for tag in airports]
        source_airports_list = airports_list[0::2]
        destination_airports_list = airports_list[1::2]

        #find the time
        times = self.driver.find_elements_by_xpath('//div[@class="section times"]/div[1]')
        times_list = [tag.text for tag in times]
        source_times_list = times_list[0::2]
        destination_times_list = times_list[1::2]
        
        #find carriers
        carriers = self.driver.find_elements_by_xpath('//div[@class="section times"]/div[2]')
        carriers_list = [tag.text for tag in carriers]
        source_carriers_list = carriers_list[0::2]
        destination_carriers_list = carriers_list[1::2]
    

        ## Store to dictionary
        kayak_dict = {}
        kayak_dict[self.fieldnames[0]]  = prices_list
        kayak_dict[self.fieldnames[1]]  = source_stops_list
        kayak_dict[self.fieldnames[2]]  = destination_stops_list
        kayak_dict[self.fieldnames[3]]  = source_airport_stops_list
        kayak_dict[self.fieldnames[4]]  = destination_airport_stops_list
        kayak_dict[self.fieldnames[5]]  = source_durations_list
        kayak_dict[self.fieldnames[6]]  = destination_durations_list
        kayak_dict[self.fieldnames[7]]  = source_airports_list
        kayak_dict[self.fieldnames[8]]  = destination_airports_list
        kayak_dict[self.fieldnames[9]]  = source_times_list
        kayak_dict[self.fieldnames[10]] = destination_times_list
        kayak_dict[self.fieldnames[11]] = source_carriers_list
        kayak_dict[self.fieldnames[12]] = destination_carriers_list
        kayak_dict[self.fieldnames[13]] = _sort
        

        print("*"*50)
        print(kayak_dict)

        return kayak_dict


    def scrape_flight(self, url):
        self.driver.get(url)
        self.closePopup()
        self.writer.writerows(self.scrape_sortings())

        
    def scrape_sortings(self):
        """ returns scraped data in 3-tuple of differents sorting methods on flight page
            in order (best, cheapest, quickest)"""
        return (self.scrape_data(), 
        self.scrape_data("cheapest"),
        self.scrape_data("quickest"))


    def closePopup(self):
        """ Close popups that appear on flight pages"""
        try:
            popUp_button = self.driver.find_elements_by_xpath('//button[contains(@id, "dialog-close") and contains(@class, "Button-No-Standard-Style close ")]')
            popUp_button[5].click()
            time.sleep(5)
        except:
            pass


    def clickQuickest(self):
        """ Helper function to change sort on page"""
        self.driver.find_element_by_xpath('//a[@data-code = "duration"]').click()
        time.sleep(5)


    def clickCheapest(self):
        """ Helper function to change sort on page"""
        self.driver.find_element_by_xpath('//a[@data-code = "price"]').click()
        time.sleep(5)


    def makeFlightURL(self,source,dest,start_date,end_date):
        return f"https://www.kayak.com/flights/{source}-{dest}/{start_date}/{end_date}?sort=bestflight_a"

In [ ]:
scraper = KayakScraper()
scraper.scrape_flight(scraper.makeFlightURL("NYC","LAX","2019-08-30","2019-09-02"))

In [ ]:
tot_dict = {}
for i in range(5):
    kayak_dict = {1:'a',2:'b',3:'c'}
    tot_dict[i] = kayak_dict

tot_dict

In [ ]:
class KayakScraper(object):
    def __init__(self):
        self.driver = webdriver.Chrome("./chromedriver/chromedriver")
        self.file = open("./data/kayak.csv",'w+')
        self.fieldnames = ['price','src_stops','dest_stops',
                    'src_airport_stops', 'dest_airport_stops',
                    'src_duration', 'dest_duration',
                    'src_airport', 'dest_airport',
                    'src_time', 'dest_time',
                    'src_carrier', 'dest_carrier',
                    'sort'
                ]

        ## Dictionary to CSV writer class
        self.writer = csv.DictWriter(self.file, fieldnames = self.fieldnames)
        ## Write column names to CSV header
        self.writer.writeheader()

        self.driver.get("https://www.kayak.com/")
        time.sleep(5)

        
    def __del__(self):
        self.driver.quit()
        del self.writer
        self.file.close()
        

    def show_more_results(self):
        try:        
            showMoreResults_button = self.driver.find_element_by_xpath('//a[@class="moreButton"]')
            showMoreResults_button
            time.sleep(10)
        except:
            pass
        

    def scrape_data(self, _sort = "best"):
        if _sort == "cheapest":
            self.clickCheapest()
        elif _sort == "quickest":
            self.clickQuickest()
            
            
        kayak_dict = {}
        
        ## Ignore "sort" field, added later
        for i in self.fieldnames[:-1]:
            kayak_dict[i] = []
            
        
        for i in range(50):
                        
            #find prices
            prices = self.waitobject('//span[@class="price option-text"]')
            kayak_dict['price'] = kayak_dict['price'] + [tag.text for tag in prices if tag.text != '']
            #print(kayak_dict['price'])
            
#             prices_list = [tag.text.replace('$','') for tag in prices if tag.text != '']
#             prices_list = list(map(int, prices_list))

            #find number of stops
            stops  = self.waitobject('//div[@class="section stops"]/div[1]')
            stops_list = [tag.text[0] for tag in stops if stops != '']
            stops_list = [tag.replace('n','0') for tag in stops_list]
            stops_list = list(map(int, stops_list))
            kayak_dict['src_stops']  = kayak_dict['src_stops'] + stops_list[0::2]
            kayak_dict['dest_stops'] = kayak_dict['dest_stops'] + stops_list[1::2]
            

            #find airport cities for stops
            airport_stops = self.waitobject('//div[@class="section stops"]/div[2]')
            airport_stops_list = [tag.text for tag in airport_stops if airport_stops != '']
            kayak_dict['src_airport_stops'] = kayak_dict['src_airport_stops'] + airport_stops_list[0::2]
            kayak_dict['dest_airport_stops'] = kayak_dict['dest_airport_stops'] + airport_stops_list[1::2]
            

            #find durations
            durations = self.waitobject('//div[@class="section duration"]/div[1]')
            durations_list = [tag.text for tag in durations]
            kayak_dict['src_duration'] = kayak_dict['src_duration']  +  durations_list[0::2]
            kayak_dict['dest_duration'] = kayak_dict['dest_duration'] +  durations_list[1::2]
            

            #find airports cities for source and destination
            airports = self.waitobject('//div[@class="section duration"]/div[2]')
            airports_list = [tag.text for tag in airports]
            kayak_dict['src_airport'] = kayak_dict['src_airport'] +  airports_list[0::2]
            kayak_dict['dest_airport'] = kayak_dict['dest_airport'] +  airports_list[1::2]
            

            #find the time
            times = self.waitobject('//div[@class="section times"]/div[1]')
            times_list = [tag.text for tag in times]
            kayak_dict['src_time'] = kayak_dict['src_time'] +  times_list[0::2]
            kayak_dict['dest_time'] = kayak_dict['dest_time'] + times_list[1::2]
            
            
            #find carriers
            carriers = self.waitobject('//div[@class="section times"]/div[2]')
            carriers_list = [tag.text for tag in carriers]
            kayak_dict['src_carrier'] = kayak_dict['src_carrier'] + carriers_list[0::2]
            kayak_dict['dest_carrier'] = kayak_dict['dest_carrier'] + carriers_list[1::2]
            
#             print("*"*50)
#             for k,v in kayak_dict.items():
#                 print(f'{k}:  {len(v)}')
            


#             ## Store to dictionary
#             kayak_dict = {}
#             kayak_dict[self.fieldnames[0]]  = prices_list
#             kayak_dict[self.fieldnames[1]]  = source_stops_list
#             kayak_dict[self.fieldnames[2]]  = destination_stops_list
#             kayak_dict[self.fieldnames[3]]  = source_airport_stops_list
#             kayak_dict[self.fieldnames[4]]  = destination_airport_stops_list
#             kayak_dict[self.fieldnames[5]]  = source_durations_list
#             kayak_dict[self.fieldnames[6]]  = destination_durations_list
#             kayak_dict[self.fieldnames[7]]  = source_airports_list
#             kayak_dict[self.fieldnames[8]]  = destination_airports_list
#             kayak_dict[self.fieldnames[9]]  = source_times_list
#             kayak_dict[self.fieldnames[10]] = destination_times_list
#             kayak_dict[self.fieldnames[11]] = source_carriers_list
#             kayak_dict[self.fieldnames[12]] = destination_carriers_list
#             kayak_dict[self.fieldnames[13]] = [_sort for i in range(len(prices_list))]
#             #print("*"*50)
#             #print(kayak_dict)
            
#             tot_dict[i].append(kayak_dict)

        return (_sort, kayak_dict)


    def waitobject(self, xpath):
        return WebDriverWait(self.driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, xpath)))

    def scrape_flight(self, url):
        self.driver.get(url)
        time.sleep(10)
        self.closePopup()
        for ordering, data in self.scrape_sortings():
            for row in pd.DataFrame(data).iterrows():
                print(row)
                temp_dict = row[1].to_dict()
                temp_dict[self.fieldnames[13]] = ordering
                self.writer.writerow(temp_dict)

    def scrape_sortings(self):
        """ returns scraped data in 3-tuple of differents sorting methods on flight page
            in order (best, cheapest, quickest)"""
        return [self.scrape_data(),
                self.scrape_data("cheapest"),
                self.scrape_data("quickest")]


    def closePopup(self):
        """ Close popups that appear on flight pages"""
        try:
            popUp_button = self.driver.find_elements_by_xpath('//button[contains(@id, "dialog-close") and contains(@class, "Button-No-Standard-Style close ")]')
            popUp_button[5].click()
            time.sleep(15)
        except:
            pass

    def clickQuickest(self):
        """ Helper function to change sort on page"""
        self.driver.find_element_by_xpath('//a[@data-code = "duration"]').click()
        time.sleep(10)

    def clickCheapest(self):
        """ Helper function to change sort on page"""
        self.driver.find_element_by_xpath('//a[@data-code = "price"]').click()
        time.sleep(10)

    def makeFlightURL(self,source,dest,start_date,end_date):
        return f"https://www.kayak.com/flights/{source}-{dest}/{start_date}/{end_date}?sort=bestflight_a"

In [ ]:
#WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@data-code = "duration"]'))).click()

In [ ]:
scraper = KayakScraper()
scraper.scrape_flight(scraper.makeFlightURL("NYC","PER","2019-09-06","2019-09-10"))